In [23]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
%reload_ext autoreload
%autoreload 2

In [1]:
from transformers import FlaxGPT2LMHeadModel

model = FlaxGPT2LMHeadModel.from_pretrained("gpt2")

/var/local/ML/TRAIN/jax/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import flax

type(model.params)
flax.traverse_util.flatten_dict(model.params).keys()

dict_keys([('transformer', 'h', '0', 'attn', 'c_attn', 'bias'), ('transformer', 'h', '0', 'attn', 'c_attn', 'kernel'), ('transformer', 'h', '0', 'attn', 'c_proj', 'bias'), ('transformer', 'h', '0', 'attn', 'c_proj', 'kernel'), ('transformer', 'h', '0', 'ln_1', 'bias'), ('transformer', 'h', '0', 'ln_1', 'scale'), ('transformer', 'h', '0', 'ln_2', 'bias'), ('transformer', 'h', '0', 'ln_2', 'scale'), ('transformer', 'h', '0', 'mlp', 'c_fc', 'bias'), ('transformer', 'h', '0', 'mlp', 'c_fc', 'kernel'), ('transformer', 'h', '0', 'mlp', 'c_proj', 'bias'), ('transformer', 'h', '0', 'mlp', 'c_proj', 'kernel'), ('transformer', 'h', '1', 'attn', 'c_attn', 'bias'), ('transformer', 'h', '1', 'attn', 'c_attn', 'kernel'), ('transformer', 'h', '1', 'attn', 'c_proj', 'bias'), ('transformer', 'h', '1', 'attn', 'c_proj', 'kernel'), ('transformer', 'h', '1', 'ln_1', 'bias'), ('transformer', 'h', '1', 'ln_1', 'scale'), ('transformer', 'h', '1', 'ln_2', 'bias'), ('transformer', 'h', '1', 'ln_2', 'scale'), (

# No dropout


In [9]:
from gpt2 import GPT
import flax.nnx as nnx

model = GPT.from_pretrained("gpt2")
model.train()
nnx.state(model).flat_state().keys()

loading weights from pretrained gpt: gpt2
Length of prepared JAX modules dict: 89


dict_keys([('drop', 'rngs', 'default', 'count'), ('drop', 'rngs', 'default', 'key'), ('h', 0, 'attn', 'c_attn', 'bias'), ('h', 0, 'attn', 'c_attn', 'kernel'), ('h', 0, 'attn', 'c_proj', 'bias'), ('h', 0, 'attn', 'c_proj', 'kernel'), ('h', 0, 'ln_1', 'bias'), ('h', 0, 'ln_1', 'scale'), ('h', 0, 'ln_2', 'bias'), ('h', 0, 'ln_2', 'scale'), ('h', 0, 'mlp', 'c_fc', 'bias'), ('h', 0, 'mlp', 'c_fc', 'kernel'), ('h', 0, 'mlp', 'c_proj', 'bias'), ('h', 0, 'mlp', 'c_proj', 'kernel'), ('h', 1, 'attn', 'c_attn', 'bias'), ('h', 1, 'attn', 'c_attn', 'kernel'), ('h', 1, 'attn', 'c_proj', 'bias'), ('h', 1, 'attn', 'c_proj', 'kernel'), ('h', 1, 'ln_1', 'bias'), ('h', 1, 'ln_1', 'scale'), ('h', 1, 'ln_2', 'bias'), ('h', 1, 'ln_2', 'scale'), ('h', 1, 'mlp', 'c_fc', 'bias'), ('h', 1, 'mlp', 'c_fc', 'kernel'), ('h', 1, 'mlp', 'c_proj', 'bias'), ('h', 1, 'mlp', 'c_proj', 'kernel'), ('h', 2, 'attn', 'c_attn', 'bias'), ('h', 2, 'attn', 'c_attn', 'kernel'), ('h', 2, 'attn', 'c_proj', 'bias'), ('h', 2, 'attn', 

# We have one ackward Dropout here that is causing the checkpointing to fail. No idea why the other Dropout modules dont have state!


In [15]:
import flax.nnx as nnx
import jax


class Inner(nnx.Module):
    def __init__(self, rngs) -> None:
        super().__init__()
        self.inner_dropout = nnx.Dropout(0.1, rngs=rngs)


class Outer(nnx.Module):
    def __init__(self, rngs) -> None:
        super().__init__()
        rngs, inner_dropout_key, outer_dropout_key = jax.random.split(rngs.key(), 3)
        self.outer_dropout = nnx.Dropout(0.2, rngs=outer_dropout_key)
        self.inner = Inner(inner_dropout_key)


outer = Outer(nnx.Rngs(0))
nnx.state(outer).flat_state()

{('inner',
  'inner_dropout',
  'rngs'): Array((), dtype=key<fry>) overlaying:
 [3099025716 3811338087],
 ('outer_dropout',
  'rngs'): Array((), dtype=key<fry>) overlaying:
 [ 862470802 3115470296]}

# Split keys appears to add state for all dropouts


In [20]:
class Inner1(nnx.Module):
    def __init__(self, rngs) -> None:
        super().__init__()
        self.inner_dropout = nnx.Dropout(0.1, rngs=rngs)


class Outer1(nnx.Module):
    def __init__(self, rngs) -> None:
        super().__init__()
        self.outer_dropout = nnx.Dropout(0.2, rngs=rngs)
        self.inner = Inner1(rngs)


outer = Outer1(nnx.Rngs(0))
nnx.state(outer).flat_state()

{('inner',
  'inner_dropout',
  'rngs',
  'default',
  'count'): VariableState(
   type=RngCount,
   value=Array(0, dtype=uint32),
   tag='default'
 ),
 ('inner',
  'inner_dropout',
  'rngs',
  'default',
  'key'): VariableState(
   type=RngKey,
   value=Array((), dtype=key<fry>) overlaying:
   [0 0],
   tag='default'
 )}

In [22]:
nnx.state(outer).flat_state()[("inner", "inner_dropout", "rngs", "default", "key")]

VariableState(
  type=RngKey,
  value=Array((), dtype=key<fry>) overlaying:
  [0 0],
  tag='default'
)

# Only one Dropout gets state, no idea how that one is chosen


In [26]:
from gpt2 import GPT
import optax
import flax.nnx as nnx
import numpy as np
import tiktoken
import datasets

model = GPT.from_pretrained("gpt2")
model.train()
tx = optax.adamw(learning_rate=1e-4, weight_decay=1e-4)
optimizer = nnx.Optimizer(model, tx)
metrics = nnx.MultiMetric(
    accuracy=nnx.metrics.Accuracy(),
    loss=nnx.metrics.Average("loss"),
)

enc = tiktoken.get_encoding("gpt2")
batch_size = 6
block_size = 1024

data = datasets.load_dataset(path="Trelis/tiny-shakespeare")
train_data = "\n".join([x["Text"] for x in data["train"]])
train_data = enc.encode_ordinary(train_data)
train_data = np.array(train_data, dtype=np.uint16)
val_data = "\n".join([x["Text"] for x in data["test"]])
val_data = enc.encode_ordinary(val_data)
val_data = np.array(val_data, dtype=np.uint16)


@nnx.jit
def train_step(model: GPT, optimizer: nnx.Optimizer, metrics: nnx.MultiMetric, batch):
    x, y = batch

    def loss_fn(model: GPT):
        logits = model(x)
        loss = optax.softmax_cross_entropy_with_integer_labels(
            logits.reshape([-1, logits.shape[-1]]), y.reshape([-1])
        ).mean()
        return loss, logits

    grad_fn = nnx.value_and_grad(loss_fn, has_aux=True)
    (loss, logits), grads = grad_fn(model)
    metrics.update(loss=loss, logits=logits, labels=y)
    optimizer.update(grads)


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = np.random.randint(len(data) - block_size, size=(batch_size,))
    x = np.stack([data[i : i + block_size].astype(np.int32) for i in ix])
    y = np.stack([data[i + 1 : i + 1 + block_size].astype(np.int32) for i in ix])
    return x, y


nnx.state(model).flat_state()[("drop", "rngs", "default", "key")]
nnx.state(model).flat_state()[("drop", "rngs", "default", "count")]
train_step(model, optimizer, metrics, get_batch("train"))
nnx.state(model).flat_state()[("drop", "rngs", "default", "key")]
nnx.state(model).flat_state()[("drop", "rngs", "default", "count")]

loading weights from pretrained gpt: gpt2


Length of prepared JAX modules dict: 89


VariableState(
  type=RngKey,
  value=Array((), dtype=key<fry>) overlaying:
  [0 0],
  tag='default'
)

VariableState(
  type=RngCount,
  value=Array(149, dtype=uint32),
  tag='default'
)

VariableState(
  type=RngKey,
  value=Array((), dtype=key<fry>) overlaying:
  [0 0],
  tag='default'
)

VariableState(
  type=RngCount,
  value=Array(162, dtype=uint32),
  tag='default'
)

# Training does not appear to modify the key
